In [ ]:
import pandas as pd
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import numpy as np
import glob
import sys
sys.path.append('/Users/david/Dropbox/PhD/Scripts/Spatial analyses')
import psycopg2
from pathlib import Path
from shapely.geometry import Point, Polygon
import geoplot
import geoplot.crs as gcrs
import datashader as ds, colorcet as cc
# import holoviews as hv
# hv.extension("bokeh")
# from holoviews.element.tiles import EsriImagery
import rasterio
from rasterio.mask import mask
from rasterio.plot import show
import fiona
import seaborn as sns
from datashader.utils import export_image
# from holoviews.operation.datashader import datashade
import matplotlib.pyplot as plt
import libpysal as lps
from scipy.spatial import cKDTree
from libpysal.weights.distance import get_points_array
from esda import fdr
import pyspace
import xgboost
import shap
from sklearn.metrics import mean_squared_error, r2_score
from importlib import reload
plt.rc('font', family='Helvetica')
from tableone import TableOne

In [ ]:
res_folder = Path('../Results/')
model_folder = res_folder/'Models'

## Determinants of using an integrative approach

In [ ]:
# data = pd.read_parquet('../Data/processed/full_dataset_nonull.parquet.gzip')
# data_2017_psm = pd.read_parquet('../Data/processed/data_2017_scaled_and_matched.parquet')

In [ ]:
# data = gpd.GeoDataFrame(data, crs = 4326, geometry=gpd.points_from_xy(data.lon_masked, data.lat_masked))

# data = data.to_crs(2056)
# data['E'], data['N'] = data['geometry'].x, data['geometry'].y

In [ ]:
# data_final['gp'] = data_final['gp'].astype(str)
# data_final = data_final.reset_index(drop = True)

In [ ]:
# del data

In [ ]:
df_treated_filtered = pd.read_parquet('../Data/processed/df_treated_filtered.parquet.gzip')

In [ ]:
df_treated_filtered.groupby('NOANNEE').size().sum()

In [ ]:
df_treated_filtered['MTFRANCHISECOUV'].unique()

In [ ]:
# data_final = data[data.treatment.isnull()==False]

df_treated_filtered['DEDUCTIBLE_above_500'] = 1
df_treated_filtered.loc[df_treated_filtered.MTFRANCHISECOUV < 500, 'DEDUCTIBLE_above_500'] = 0

### Add chronic diseases info

In [ ]:
# data_final[data_final['Diabetes mellitus'] > 0].uuid.nunique()

In [ ]:
# # Reverse the dictionary for grouping by disease
# disease_to_atc = {}
# for atc, disease in dict_chronic_diseases_atc_amount.items():
#     if disease in disease_to_atc:
#         disease_to_atc[disease].append(atc)
#     else:
#         disease_to_atc[disease] = [atc]

# # For each disease, create a new column and check if any ATC columns are 1
# for disease, atcs in disease_to_atc.items():
#     data_final[disease] = data_final[atcs].sum(axis=1)

# # If you want binary 0/1 values
# # data_final = data_final.replace({disease: {2: 1 for disease in disease_to_atc.keys()}})


### 1. Baseline model - Logistic regression

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
df_treated_filtered[['NBAGE_std','ssep3_std','cds_std','E_std','N_std','D_MEDIC_B_std','D_MEDIC_S_std']] = scaler.fit_transform(df_treated_filtered[['NBAGE','ssep3','cds','E','N','D_MEDIC_B','D_MEDIC_S']])

In [ ]:
# data_final['CANTON'] = data_final.filter(regex='SEX_').idxmax(axis=1).str.replace('CANTON_NAME_', '')
# data_final['SEX'] = data_final.filter(regex='SEX_').idxmax(axis=1).str.replace('SEX_', '')
# data_final['LANG'] = data_final.filter(regex='LANG_').idxmax(axis=1).str.replace('LANG_', '')

In [ ]:
# df_treated_filtered['NBRE_FACTURES_AOS'] = df_treated_filtered['NBRE_FACTURES_AOS'].astype(float)
df_treated_filtered['n_atc'] = df_treated_filtered['n_atc'].astype(int)

In [ ]:
# data_2017 = data_final[data_final.NOANNEE == 2017]
# data_2018 = data_final[data_final.NOANNEE == 2018]
# data_2019 = data_final[data_final.NOANNEE == 2019]
# data_2020 = data_final[data_final.NOANNEE == 2020]
# data_2021 = data_final[data_final.NOANNEE == 2021]

In [ ]:
dict_chronic_diseases_atc = {
    "A02A_ATC_N":'Acid related disorders',
    "A02B_ATC_N":'Acid related disorders',
    'M05B_ATC_N':'Osteoporosis',
    'A10A_ATC_N':'Diabetes mellitus',
    'A10B_ATC_N':'Diabetes mellitus',
    'B03A_ATC_N':'Iron deficiency anemia',
    'N03A_ATC_N': 'Epilepsy',
    'C02A_ATC_N':'Cardiovascular diseases',
    'C02C_ATC_N':'Cardiovascular diseases',
    'C02D_ATC_N':'Cardiovascular diseases',
    'C02K_ATC_N':'Cardiovascular diseases',
    'C04A_ATC_N':'Cardiovascular diseases',
    'C07A_ATC_N':'Cardiovascular diseases',
    'C08C_ATC_N':'Cardiovascular diseases',
    'C08D_ATC_N':'Cardiovascular diseases',
    'C09A_ATC_N':'Cardiovascular diseases',
    'C09B_ATC_N':'Cardiovascular diseases',
    'C09C_ATC_N':'Cardiovascular diseases',
    'C09D_ATC_N':'Cardiovascular diseases',
    "C10A_ATC_N":"Hyperlipidemia",
    "C10B_ATC_N":"Hyperlipidemia",
    'L01A_ATC_N':'Cancer',
    'L01B_ATC_N':'Cancer',
    'L01C_ATC_N':'Cancer',
    'L01D_ATC_N':'Cancer',
    'L01E_ATC_N':'Cancer',
    'L01F_ATC_N':'Cancer',
    'L01X_ATC_N':'Cancer',
    "M01A_ATC_N":'Rheumatologic conditions',
    "M01C_ATC_N":'Rheumatologic conditions',
    "M02A_ATC_N":'Rheumatologic conditions',
    "M04A_ATC_N":'Gout and hyperuricemia',
    'N02A_ATC_N':'Pain',
    'N02B_ATC_N':"Pain",
    "N02C_ATC_N":'Migraines',
    "N04A_ATC_N":"Parkinson's disease",
    "N04B_ATC_N":"Parkinson's disease",
    'N05A_ATC_N':'Psychoses',

    'N05B_ATC_N':'Psychological disorders',
    'N05C_ATC_N':'Psychological disorders',
    'N06A_ATC_N':'Psychological disorders',

    "N06D_ATC_N":'Dementia',
    "L04A_ATC_N":'Rheumatologic conditions',
    "H03A_ATC_N":'Thyroid disorders',
    "J04A_ATC_N":'Tuberculosis',
    'J05A_ATC_N':'HIV',
    "R03A_ATC_N":'Respiratory illness',
    "S01E_ATC_N":'Glaucoma'
}

dict_chronic_diseases_atc_amount = {
    "A02A_ATC_AMOUNT":'Acid related disorders',
    "A02B_ATC_AMOUNT":'Acid related disorders',
    'M05B_ATC_AMOUNT':'Osteoporosis',
    'A10A_ATC_AMOUNT':'Diabetes mellitus',
    'A10B_ATC_AMOUNT':'Diabetes mellitus',
    'B03A_ATC_AMOUNT':'Iron deficiency anemia',
    'N03A_ATC_AMOUNT': 'Epilepsy',
    'C02A_ATC_AMOUNT':'Cardiovascular diseases',
    'C02C_ATC_AMOUNT':'Cardiovascular diseases',
    'C02D_ATC_AMOUNT':'Cardiovascular diseases',
    'C02K_ATC_AMOUNT':'Cardiovascular diseases',
    'C04A_ATC_AMOUNT':'Cardiovascular diseases',
    'C07A_ATC_AMOUNT':'Cardiovascular diseases',
    'C08C_ATC_AMOUNT':'Cardiovascular diseases',
    'C08D_ATC_AMOUNT':'Cardiovascular diseases',
    'C09A_ATC_AMOUNT':'Cardiovascular diseases',
    'C09B_ATC_AMOUNT':'Cardiovascular diseases',
    'C09C_ATC_AMOUNT':'Cardiovascular diseases',
    'C09D_ATC_AMOUNT':'Cardiovascular diseases',
    "C10A_ATC_AMOUNT":"Hyperlipidemia",
    "C10B_ATC_AMOUNT":"Hyperlipidemia",
    'L01A_ATC_AMOUNT':'Cancer',
    'L01B_ATC_AMOUNT':'Cancer',
    'L01C_ATC_AMOUNT':'Cancer',
    'L01D_ATC_AMOUNT':'Cancer',
    'L01E_ATC_AMOUNT':'Cancer',
    'L01F_ATC_AMOUNT':'Cancer',
    'L01X_ATC_AMOUNT':'Cancer',
    "M01A_ATC_AMOUNT":'Rheumatologic conditions',
    "M01C_ATC_AMOUNT":'Rheumatologic conditions',
    "M02A_ATC_AMOUNT":'Rheumatologic conditions',
    "M04A_ATC_AMOUNT":'Gout and hyperuricemia',
    'N02A_ATC_AMOUNT':'Pain',
    'N02B_ATC_AMOUNT':"Pain",
    "N02C_ATC_AMOUNT":'Migraines',
    "N04A_ATC_AMOUNT":"Parkinson's disease",
    "N04B_ATC_AMOUNT":"Parkinson's disease",
    'N05A_ATC_AMOUNT':'Psychoses',
    'N05B_ATC_AMOUNT':'Psychological disorders',
    'N05C_ATC_AMOUNT':'Psychological disorders',
    'N06A_ATC_AMOUNT':'Psychological disorders',
    "N06D_ATC_AMOUNT":'Dementia',
    "L04A_ATC_AMOUNT":'Rheumatologic conditions',
    "H03A_ATC_AMOUNT":'Thyroid disorders',
    "J04A_ATC_AMOUNT":'Tuberculosis',
    'J05A_ATC_AMOUNT':'HIV',
    "R03A_ATC_AMOUNT":'Respiratory illness',

    "S01E_ATC_AMOUNT":'Glaucoma',
}

In [ ]:
chronic_diseases = list(set(dict_chronic_diseases_atc.values()))
chronic_diseases_binary = [s + '_binary' for s in chronic_diseases]

In [ ]:
exog_vars = ['NBAGE_std','SEX_F', 'ssep3_std','cds', 'DEDUCTIBLE_above_500', 'E', 'N']
exog = sm.add_constant(df_treated_filtered[exog_vars])
endog = df_treated_filtered['treatment']

In [ ]:
' + '.join(df_treated_filtered.filter(regex='region_').columns)

In [ ]:
# Define the logistic regression formula using your covariates
formula = "treatment ~ NBAGE_std + SEX_F + region_DE + region_FR + region_IT + ssep3_std + urb_Urbain + urb_Périurbain + cds_std + D_MEDIC_B_std + D_MEDIC_S_std + DEDUCTIBLE_above_500 + Asthma_PCG + Cancer_PCG + Diabetes_PCG + Epilepsy_PCG + Glaucoma_PCG + HIV_AIDS_PCG + Heart_disease_PCG + Hypertension_related_PCG + Immune_PCG + Inflammatory_PCG + Mental_PCG + Other_PCG + Pain_PCG + Parkinson_PCG + Thyroid_PCG + mean_ndvi + mean_lst + mean_pm10 + mean_pm25 + mean_no2 + mean_carnight + E_std + N_std + E_std*N_std"

# Fit the logistic regression model using statsmodels
logit_model = smf.logit(formula, data=df_treated_filtered).fit()

# Print the model summary
print(logit_model.summary())

In [ ]:
df_treated_filtered.filter(regex='PCG').columns.str.replace('_PCG','')

In [ ]:
df_treated_filtered.filter(regex='region').columns

In [ ]:
len(['NBAGE',"NBAGE_std", "ssep3_std",'ssep3','region_DE', 'region_FR', 'region_IT','Asthma_PCG', 'Cancer_PCG', 'Diabetes_PCG', 'Epilepsy_PCG',
       'Glaucoma_PCG', 'HIV_AIDS_PCG', 'Heart_disease_PCG',
       'Hypertension_related_PCG', 'Immune_PCG', 'Inflammatory_PCG',
       'Mental_PCG', 'Other_PCG', 'Pain_PCG', 'Parkinson_PCG', 'Thyroid_PCG', "SEX_F",'SEX','LANG', "cds_std",'cds','LANG_FR','D_MEDIC_B','D_MEDIC_S','D_MEDIC_B_std','D_MEDIC_S_std','DEDUCTIBLE_above_500','E_std','N_std','E_std:N_std','PRESTATIONS_TOTAL','PRESTATIONS_BRUTES_AOS','PRESTATIONS_BRUTES_LCA','AMBULATOIRE','STATIONNAIRE','PRESTATIONS_ACCIDENT','PRESTATIONS_DISEASE','PRESTATIONS_BIRTH','MTFRANCHISECOUV','mean_pm10','mean_no2','mean_pm25','mean_ndvi','mean_lst','mean_carnight'])

In [ ]:
len(['Age',"Age", "SES index",'SES index''Asthma', 'Cancer', 'Diabetes', 'Epilepsy', 'Glaucoma', 'HIV_AIDS',
       'Heart_disease', 'Hypertension_related', 'Immune', 'Inflammatory',
       'Mental', 'Other', 'Pain', 'Parkinson', 'Thyroid', "Sex (Female)",'Sex','Langage', "CDS",'CDS','French speaker','Access to prim. care med.','Access to spec. med.','Access to prim. care med.','Access to spec. med.','Franchise (>500)','E','N','E:N','Montant tot. prestations','Montant tot. prestations (AOS)','Montant tot. prestations (LCA)','Montant tot. ambulatoire','Montant tot. stationnaire','Montant tot. accident','Montant tot. maladie','Montant tot. maternité','Franchise','PM10','NO2','PM25','NDVI','LST','Nighttime car noise'])

In [ ]:
variable_names = pd.DataFrame({"old": ['NBAGE',"NBAGE_std", "ssep3_std",'ssep3', 'region_DE', 'region_FR', 'region_IT','urb_Urbain','urb_Périurbain','Asthma_PCG', 'Cancer_PCG', 'Diabetes_PCG', 'Epilepsy_PCG',
       'Glaucoma_PCG', 'HIV_AIDS_PCG', 'Heart_disease_PCG',
       'Hypertension_related_PCG', 'Immune_PCG', 'Inflammatory_PCG',
       'Mental_PCG', 'Other_PCG', 'Pain_PCG', 'Parkinson_PCG', 'Thyroid_PCG', "SEX_F",'SEX','LANG', "cds_std",'cds','LANG_FR','D_MEDIC_B','D_MEDIC_S','D_MEDIC_B_std','D_MEDIC_S_std','DEDUCTIBLE_above_500','E_std','N_std','E_std:N_std','PRESTATIONS_TOTAL','PRESTATIONS_BRUTES_AOS','PRESTATIONS_BRUTES_LCA','AMBULATOIRE','STATIONNAIRE','PRESTATIONS_ACCIDENT','PRESTATIONS_DISEASE','PRESTATIONS_BIRTH','MTFRANCHISECOUV','mean_pm10','mean_no2','mean_pm25','mean_ndvi','mean_lst','mean_carnight'],
                           "new": ['Age',"Age", "SES index",'SES index','region_DE', 'region_FR', 'region_IT','Urban','Periurban', 'Asthma', 'Cancer', 'Diabetes', 'Epilepsy', 'Glaucoma', 'HIV/AIDS',
       'Heart disease', 'Hypertension related', 'Immune', 'Inflammatory',
       'Mental', 'Other', 'Pain', 'Parkinson', 'Thyroid', "Sex (Female)",'Sex','Langage', "CDS",'CDS','French speaker','Access to prim. care med.','Access to spec. med.','Access to prim. care med.','Access to spec. med.','Franchise (>500)','E','N','E:N','Montant tot. prestations','Montant tot. prestations (AOS)','Montant tot. prestations (LCA)','Montant tot. ambulatoire','Montant tot. stationnaire','Montant tot. accident','Montant tot. maladie','Montant tot. maternité','Franchise','PM10','NO2','PM25','NDVI','LST','Nighttime car noise']})
def update_variable_names(summary_table, variable_names, table_type):
    name_mapper = variable_names.set_index('old')['new'].to_dict()
    if table_type == 'summary':
        name_mapper = {f"{key}, mean (SD)": f"{value}, mean (SD)" for key, value in name_mapper.items()}

    summary_table = summary_table.rename(index=name_mapper)
    return summary_table

In [ ]:
params = logit_model.params
conf = logit_model.conf_int()
conf['Odds Ratio'] = params
conf.columns = ['2.5%', '97.5%', 'Odds Ratio']
# convert log odds to ORs
odds = pd.DataFrame(np.exp(conf))
# check if pvalues are significant
odds['pvalues'] = logit_model.pvalues
odds['significant?'] = ['significant' if pval <= 0.05 else 'not significant' for pval in logit_model.pvalues]
updated_odds = update_variable_names(odds, variable_names, 'odds')
updated_odds

In [ ]:
def forest_plot(df_odds, folder, period):

    if not os.path.exists(folder):
        os.makedirs(folder)
    fig, ax = plt.subplots(nrows=1, sharex=True, sharey=True, figsize=(6, 6), dpi=300)
    for idx, row in df_odds.iloc[::-1].iterrows():
        ci = [[row['Odds Ratio'] - row[::-1]['2.5%']], [row['97.5%'] - row['Odds Ratio']]]
        if row['significant?'] == 'significant':
            if row['Odds Ratio'] > 1:
                plt.errorbar(x=[row['Odds Ratio']], y=[row.name], xerr=ci,
                    ecolor='tab:red', capsize=3, linestyle='None', linewidth=1, marker="o", 
                             markersize=5, mfc="tab:red", mec="tab:red")
            else:
                plt.errorbar(x=[row['Odds Ratio']], y=[row.name], xerr=ci,
                    ecolor='tab:blue', capsize=3, linestyle='None', linewidth=1, marker="o", 
                             markersize=5, mfc="tab:blue", mec="tab:blue")
        else:
            plt.errorbar(x=[row['Odds Ratio']], y=[row.name], xerr=ci,
                ecolor='tab:gray', capsize=3, linestyle='None', linewidth=1, marker="o", 
                         markersize=5, mfc="tab:gray", mec="tab:gray")
    plt.axvline(x=1, linewidth=0.8, linestyle='--', color='black')
    plt.tick_params(axis='both', which='major', labelsize=8)
    plt.xlabel('Odds Ratio and 95% Confidence Interval', fontsize=8)
    plt.tight_layout()
    plt.savefig(folder/'forest_plot_{}.png'.format(period))
    plt.show()
model_directory = res_folder /'Determinants of integrative medicine'/ 'Logistic regression - Full period'
forest_plot(updated_odds.sort_values('Odds Ratio'), model_directory, 'Full')

In [ ]:
columns = ['PRESTATIONS_TOTAL','PRESTATIONS_BRUTES_AOS','PRESTATIONS_BRUTES_LCA','D_MEDIC_S','D_MEDIC_B','AMBULATOIRE','STATIONNAIRE','PRESTATIONS_ACCIDENT','PRESTATIONS_DISEASE','PRESTATIONS_BIRTH']
mytable = TableOne(data_final, columns, groupby='NOANNEE', 
                   pval=True,
                   pval_adjust='bonferroni', 
                   htest_name=True, 
                   missing=False)
summary_table_year = update_variable_names(mytable.tableone,variable_names,'summary')
summary_table_year

In [ ]:
columns = ['PRESTATIONS_TOTAL','PRESTATIONS_BRUTES_AOS','PRESTATIONS_BRUTES_LCA','D_MEDIC_S','D_MEDIC_B','AMBULATOIRE','STATIONNAIRE','PRESTATIONS_ACCIDENT','PRESTATIONS_DISEASE','PRESTATIONS_BIRTH']
mytable = TableOne(data_final, columns, groupby='gp', 
                   pval=True,
                   pval_adjust='bonferroni', 
                   htest_name=True, 
                   missing=False)
summary_table_gp = update_variable_names(mytable.tableone, variable_names, 'summary')
summary_table_gp

In [ ]:
columns = ['PRESTATIONS_TOTAL','PRESTATIONS_BRUTES_AOS','PRESTATIONS_BRUTES_LCA','treatment','D_MEDIC_S','D_MEDIC_B','AMBULATOIRE','STATIONNAIRE','PRESTATIONS_ACCIDENT','PRESTATIONS_DISEASE','PRESTATIONS_BIRTH']
mytable = TableOne(data_final, columns, groupby=['LANG'], 
                   pval=True,
                   pval_adjust='bonferroni', 
                   htest_name=True, 
                   missing=False)
summary_table_lang = update_variable_names(mytable.tableone, variable_names, 'summary')
summary_table_lang

In [ ]:
categorical = ['SEX','LANG']
columns = ['NBAGE','SEX','LANG','MTFRANCHISECOUV','ssep2','cds','PRESTATIONS_TOTAL','PRESTATIONS_BRUTES_AOS','PRESTATIONS_BRUTES_LCA','AMBULATOIRE','STATIONNAIRE','D_MEDIC_S','D_MEDIC_B','PRESTATIONS_ACCIDENT','PRESTATIONS_DISEASE','PRESTATIONS_BIRTH']
mytable = TableOne(data_final, columns, categorical, groupby='gp', 
                   pval=True, 
                   pval_adjust='bonferroni', 
                   htest_name=True, 
                   missing=False, 
                   row_percent=True,
                   tukey_test=True,
                   normal_test=True)
summary_table_1 = update_variable_names(mytable.tableone, variable_names, 'summary')
summary_table_1

In [ ]:
data_final['n_chronic_diseases'] = data_final[chronic_diseases_binary].sum(axis = 1)

In [ ]:
data_final.loc[data_final['n_chronic_diseases'] > 8,'n_chronic_diseases'] = '9+'

In [ ]:
data_final['n_chronic_diseases'] = data_final['n_chronic_diseases'].astype(str)

In [ ]:
categorical = ['n_chronic_diseases']
columns = ['n_chronic_diseases']
mytable = TableOne(data_final[data_final.gp.isin(['AOS only','LCA & AOS'])], columns, categorical, groupby='gp', 
                   pval=True, 
                   pval_adjust='bonferroni', 
                   htest_name=True, 
                   missing=False, 
                   row_percent=True,
                   tukey_test=True,
                   normal_test=True)
summary_table_1 = mytable.tableone
summary_table_1

In [ ]:
categorical = chronic_diseases_binary
columns = chronic_diseases_binary
mytable = TableOne(data_final[data_final.gp.isin(['AOS only','LCA & AOS'])], columns, categorical, groupby='gp', 
                   pval=True, 
                   pval_adjust='bonferroni', 
                   htest_name=True, 
                   missing=False, 
                   row_percent=True,
                   tukey_test=True,
                   normal_test=True)
summary_table_1 = mytable.tableone
summary_table_1

## Chronic diseases vs treatment

In [ ]:
# Assuming df is your DataFrame with 'complementary_spend' and 'conventional_spend_for_disease_X'
data_final['intercept'] = 1  # Add an intercept term

# Specify the multiple linear regression model
logit_model = sm.Logit(data_final['treatment'], data_final[chronic_diseases_binary].rename(
    columns=lambda x: x.replace("_binary", "")
    )
)

# Fit the model
result = logit_model.fit()

# Summary of the model
print(result.summary())

params = result.params
conf = result.conf_int()
conf['Odds Ratio'] = params
conf.columns = ['2.5%', '97.5%', 'Odds Ratio']
# convert log odds to ORs
odds = pd.DataFrame(np.exp(conf))
# check if pvalues are significant
odds['pvalues'] = result.pvalues
odds['significant?'] = ['significant' if pval <= 0.05 else 'not significant' for pval in result.pvalues]

model_directory = res_folder /'Determinants of integrative medicine'/ 'Logistic regression - Chronic diseases - Full period'
forest_plot(odds.sort_values('Odds Ratio'), model_directory, 'Chronic diseases')

## Association between amount spent and diseases

In [ ]:
def plot_OLS_coefficients(result, var):
    # Get coefficients and confidence intervals
    coefficients = result.params
    conf_int = result.conf_int()
    conf_int['coef'] = coefficients
    conf_int['pvalues'] = result.pvalues
    conf_int['significant?'] = ['significant' if pval <= 0.05 else 'not significant' for pval in result.pvalues]
    
    # Sort by coefficients
    sorted_coef = conf_int.sort_values(by='coef')
    plt.figure(figsize=(6, 6))
    
    for idx, row in sorted_coef.iterrows():
        if idx == 'intercept':
            pass
        else:
            ci_lower = row[0]  # Lower bound of confidence interval
            ci_upper = row[1]  # Upper bound of confidence interval
            coef = row['coef']  # Coefficient value

            ci = [[coef - ci_lower], [ci_upper - coef]]

            color = 'tab:red' if row['significant?'] == 'significant' and coef > 0 else \
                    'tab:blue' if row['significant?'] == 'significant' and coef <= 0 else \
                    'tab:gray'

            plt.errorbar(x=[coef], y=[idx], xerr=ci, ecolor=color, capsize=3, linestyle='None', 
                         linewidth=1, marker="o", markersize=5, mfc=color, mec=color)

    plt.axvline(x=0, color='grey', linestyle='--')
#     plt.yticks(np.arange(len(sorted_coef)), sorted_coef.index)
    plt.xlabel('Coefficient Value (CHF)')
    plt.ylabel('Coefficient Name')
    plt.title('OLS Coefficient Plot - {}'.format(var))
    plt.show()

In [ ]:
data_final[data_final.gp == 'LCA & AOS']['Methodes de massage_amount'].plot.kde()

In [ ]:
def run_OLS_and_plot(df, dependent_var, independent_vars):
    df['intercept'] = 1  # Add an intercept term
    df[dependent_var].fillna(0, inplace=True)
    
    # Rename columns to remove '_binary'
    renamed_independent_vars = [x.replace("_binary", "") for x in independent_vars]
    df_renamed = df[independent_vars + ['intercept']].rename(columns=dict(zip(independent_vars, renamed_independent_vars)))

    # Specify the multiple linear regression model
    linear_model = sm.OLS(df[dependent_var], df_renamed)

    # Fit the model
    result = linear_model.fit()

    # Plot the coefficients
    plot_OLS_coefficients(result, var.replace("_amount",''))  # Assuming plot_OLS_coefficients function is defined

# Variables to consider
variables = ['PRESTATIONS_BRUTES_CAM',
    'Methodes ayurvediques_amount',
    "Methodes d'art therapie_amount",
    'Methodes de massage_amount',
    'Methodes energetiques_amount',
    'Methodes energetiques manuelles_amount',
    'Methodes hydrotherapeutiques_amount',
    'Methodes occidentales_amount',
    'Methodes orientales_amount',
    'Methodes prescriptives_amount',
    'Methodes psychologiques complementaires_amount',
    'Methodes reflexes_amount',
    'Methodes therapeutiques par le mouvement_amount'
]

# Assuming 'data_final' is your DataFrame and 'chronic_diseases_binary' contains your independent variables
for var in variables:
    run_OLS_and_plot(data_final[data_final.gp == 'LCA & AOS'], var, chronic_diseases_binary)

In [ ]:
from sklearn.utils import resample

def validate_inputs(df, dependent_var, independent_vars):
    if not isinstance(df, pd.DataFrame):
        raise TypeError("Input should be a Pandas DataFrame")
    if dependent_var not in df.columns:
        raise ValueError(f"{dependent_var} is not in DataFrame")
    for var in independent_vars:
        if var not in df.columns:
            raise ValueError(f"{var} is not in DataFrame")

def fit_logit_model(df, dependent_var, independent_vars):
    try:
        logit_model = sm.Logit((df[dependent_var] > 0).astype(int), df[independent_vars])
        
        return logit_model.fit(disp=0)
    except Exception as e:
        raise ValueError(f"Logistic Regression failed: {e}")

def fit_ols_model(df, dependent_var, independent_vars):
    try:
        ols_model = sm.OLS(df[dependent_var], df[independent_vars])
        return ols_model.fit(disp=0)
    except Exception as e:
        raise ValueError(f"OLS failed: {e}")

def two_part_model_bootstrap(df, dependent_var, independent_vars, n_iterations=10):
    """
    This function performs bootstrap resampling to create two-part models.
    
    Parameters:
    df: DataFrame
    dependent_var: str
    independent_vars: list
    n_iterations: int
    
    Returns:
    bootstrapped_predictions: list
    """
    validate_inputs(df, dependent_var, independent_vars)
    
    # Immutable copy to avoid altering input
    formatted_vars = list(independent_vars) + ['intercept']
    
    bootstrapped_predictions = []
    bootstrapped_logit_coef = []
    bootstrapped_ols_coef = []
    boostrapped_logit_incurring_costs = []
    for i in range(n_iterations):
        print('Iteration number {}'.format(i))
        df_sample = df.sample(frac=1, replace=True)
        df_sample['intercept'] = 1
        df_sample.reset_index(drop=True, inplace=True)

        # Logistic regression
        logit_result = fit_logit_model(df_sample, dependent_var, formatted_vars)
        bootstrapped_logit_coef.append(logit_result.params)

        # OLS regression
        ols_result = fit_ols_model(df_sample.loc[df_sample[dependent_var] > 0], dependent_var, formatted_vars)
        bootstrapped_ols_coef.append(ols_result.params)

        # Predictions
        final_prediction = logit_result.predict(df_sample[formatted_vars]) * ols_result.predict(df_sample[formatted_vars])
        bootstrapped_predictions.append(final_prediction)
        boostrapped_logit_incurring_costs.append(logit_result.predict(df_sample[formatted_vars]))
        
    return {'bootstrapped_predictions': bootstrapped_predictions, 'bootstrapped_ols_coef':bootstrapped_ols_coef,
           'bootstrapped_logit_coef':bootstrapped_logit_coef,'boostrapped_logit_incurring_costs':boostrapped_logit_incurring_costs, 'ols_result':ols_result}

# Usage
dependent_var = 'Methodes de massage_amount'
independent_vars = chronic_diseases_binary

result = two_part_model_bootstrap(data_final, dependent_var, independent_vars)

In [ ]:
chronic_diseases_binary

In [ ]:
data_final['Migraines_binary'].value_counts()

In [ ]:
# Create two-part regression models WITHOUT TRAIN AND TEST SETS
X = data_final[['Rheumatologic conditions_binary','Migraines_binary','Psychoses_binary','Epilepsy_binary','Hyperlipidemia_binary']]
y = data_final[dependent_var]
# Part 1: Logistic regression to determine the probability of incurring health care costs per patient/year
log_reg = LogisticRegression(max_iter=1000)
# X_train, X_test, y_train, y_test = train_test_split(data_2017[predictors + chronic_conditions], data_2017['PRESTATIONS_TOTAL'], test_size=0.2, random_state=42)
log_reg.fit(X, y > 0)
print(log_reg.intercept_, log_reg.coef_, log_reg.score(X, y>0))

prob_incurring_costs = log_reg.predict_proba(X)[:, 1]

In [ ]:
X_costs = X[y > 0]
y_costs = y[y > 0]
glm_gamma = sm.GLM(endog = y_costs, exog=X_costs, family=sm.families.Gamma(link=sm.families.links.Log()))
glm_gamma_results = glm_gamma.fit()
predicted_costs = glm_gamma_results.predict(X)

In [ ]:
predicted_costs.sort_values()

In [ ]:
X.loc[71322]

In [ ]:
expected_total_costs = prob_incurring_costs * predicted_costs


In [ ]:
expected_total_costs.sort_values().sum()

In [ ]:
dependent_var = 'Methodes de massage_amount'
independent_vars = ['Rheumatologic conditions_binary']


In [ ]:
glm_gamma = sm.GLM(endog = y_costs, exog=X_costs, family=sm.families.Gamma())


In [ ]:
glm_gamma_results = glm_gamma.fit()
predicted_costs = glm_gamma_results.predict(X)

In [ ]:
expected_total_costs = prob_incurring_costs * predicted_costs

In [ ]:
expected_total_costs.plot.kde()

In [ ]:
from sklearn.linear_model import LogisticRegression
data_final[dependent_var].fillna(0, inplace=True)

X = data_final[independent_vars]
y = data_final[dependent_var]
# Part 1: Logistic regression to determine the probability of incurring health care costs per patient/year
log_reg = LogisticRegression(max_iter=1000)
# X_train, X_test, y_train, y_test = train_test_split(data_2017[predictors + chronic_conditions], data_2017['PRESTATIONS_TOTAL'], test_size=0.2, random_state=42)
log_reg.fit(X, y > 0)
print(log_reg.intercept_, log_reg.coef_, log_reg.score(X, y>0))

prob_incurring_costs = log_reg.predict_proba(X)[:, 1]
#     df_cds_year['prob_cost'] = prob_incurring_costs
# Part 2: Generalized linear regression with a gamma error distribution and linear link function
# to estimate annual health care expenditures for patients incurring costs higher than 0
data_final['const'] = 1
data_final[independent_vars + ['const']]
X_costs = X[y > 0]
y_costs = y[y > 0]

glm_gamma = sm.GLM(endog = y_costs, exog=X_costs, family=sm.families.Gamma(link=sm.families.links.Identity()))
glm_gamma_results = glm_gamma.fit()
predicted_costs = glm_gamma_results.predict(X)
expected_total_costs = prob_incurring_costs * predicted_costs

In [ ]:
print(y_costs.isna().sum())
print(X_costs.isna().sum().sum())

In [ ]:
df_sample = data_final.sample(100000)

In [ ]:
bootstrapped_predictions = pd.DataFrame(result['bootstrapped_predictions'])

In [ ]:
# Turn lists of bootstrapped coefficients into DataFrames
bootstrapped_logit_coef_df = pd.DataFrame(result['bootstrapped_logit_coef'])
bootstrapped_ols_coef_df = pd.DataFrame(result['bootstrapped_ols_coef'])
bootstrapped_predictions = pd.DataFrame(result['bootstrapped_predictions'])

# Calculate 95% confidence intervals for logistic regression
logit_ci_lower = bootstrapped_logit_coef_df.quantile(0.025)
logit_ci_upper = bootstrapped_logit_coef_df.quantile(0.975)

# Calculate 95% confidence intervals for OLS regression
ols_ci_lower = bootstrapped_ols_coef_df.quantile(0.025)
ols_ci_upper = bootstrapped_ols_coef_df.quantile(0.975)

# Compute prediction intervals
prediction_interval_lower = bootstrapped_predictions.quantile(0.025, axis=1)
prediction_interval_upper = bootstrapped_predictions.quantile(0.975, axis=1)

# Let's say original_predictions are the predictions from the original model
original_predictions = np.mean(bootstrapped_predictions, axis=1)


# # Print or store these values for interpretation
# print("Logit Coefficients 95% CI:", logit_ci_lower, logit_ci_upper)
# print("OLS Coefficients 95% CI:", ols_ci_lower, ols_ci_upper)

In [ ]:
# Making the plot
plt.figure(figsize=(10, 6))
plt.scatter(range(len(original_predictions)), original_predictions, color='blue', label='Predictions')
plt.fill_between(range(len(original_predictions)), prediction_interval_lower, prediction_interval_upper, color='grey', alpha=0.5, label='95% Prediction Interval')
plt.xlabel('Observation')
plt.ylabel('Predicted Value')
plt.title('Prediction Intervals for Bootstrapped Predictions')
plt.legend()
plt.show()

In [ ]:
# Assuming bootstrapped_predictions is a DataFrame where each column is a set of predictions from a bootstrap sample
bootstrapped_predictions = pd.DataFrame(bootstrapped_predictions)

# Compute prediction intervals
prediction_interval_lower = bootstrapped_predictions.quantile(0.025, axis=1)
prediction_interval_upper = bootstrapped_predictions.quantile(0.975, axis=1)

# Let's say original_predictions are the predictions from the original model
original_predictions = np.mean(bootstrapped_predictions, axis=1)

# Making the plot
plt.figure(figsize=(10, 6))
plt.scatter(range(len(original_predictions)), original_predictions, color='blue', label='Predictions')
plt.fill_between(range(len(original_predictions)), prediction_interval_lower, prediction_interval_upper, color='grey', alpha=0.5, label='95% Prediction Interval')
plt.xlabel('Observation')
plt.ylabel('Predicted Value')
plt.title('Prediction Intervals for Bootstrapped Predictions')
plt.legend()
plt.show()

In [ ]:
def two_part_model_bootstrap_with_coef(df, dependent_var, independent_vars, n_iterations=100):
    bootstrapped_predictions = []
    bootstrapped_logit_coef = []
    bootstrapped_ols_coef = []
    
    for i in range(n_iterations):
        # Sample with replacement from original data
        df_sample = resample(df, replace=True)
        
        # Prepare independent variables
        df_sample['intercept'] = 1
        renamed_independent_vars = [x.replace("_binary", "") for x in independent_vars]
        df_renamed = df_sample[independent_vars + ['intercept']].rename(columns=dict(zip(independent_vars, renamed_independent_vars)))
        
        # Part 1: Logistic regression for zero vs. non-zero outcome
        logit_model = sm.Logit((df_sample[dependent_var] > 0).astype(int), df_renamed)
        logit_result = logit_model.fit(disp=0)
        bootstrapped_logit_coef.append(logit_result.params)
        
        # Part 2: OLS for non-zero outcomes
        ols_model = sm.OLS(df_sample.loc[df_sample[dependent_var] > 0, dependent_var], df_renamed.loc[df_sample[dependent_var] > 0])
        ols_result = ols_model.fit(disp=0)
        bootstrapped_ols_coef.append(ols_result.params)
        
        # Predictions
        prob_non_zero = logit_result.predict(df_renamed)
        ols_prediction = ols_result.predict(df_renamed)
        
        # Combined prediction
        final_prediction = prob_non_zero * ols_prediction
        bootstrapped_predictions.append(final_prediction)
        
    return bootstrapped_predictions, bootstrapped_logit_coef, bootstrapped_ols_coef

# Example usage
dependent_var = 'PRESTATIONS_BRUTES_CAM'
independent_vars = chronic_diseases_binary
bootstrapped_predictions, bootstrapped_logit_coef, bootstrapped_ols_coef = two_part_model_bootstrap_with_coef(data_final, dependent_var, independent_vars)


In [ ]:
data_folder  = Path('../Data/')

df_amount_by_lca_discipline = pd.read_parquet(data_folder/'processed'/'Intermediate datasets'/'df_amount_by_lca_discipline.parquet.gzip')

## Association between disease apparition and complementary medicine usage

In [ ]:
df_sample_uuid = data_final.head(5000).uuid.unique()
df_sample = data_final[data_final.uuid.isin(df_sample_uuid)]

In [ ]:
df_sample['Pain_diff'] = df_sample.sort_values(['uuid','NOANNEE']).groupby('uuid')['Pain_binary'].diff(1)
df_sample['CDS_diff'] = df_sample.sort_values(['uuid','NOANNEE']).groupby('uuid')['cds'].diff(1)

df_sample['Psy_diff'] = df_sample.sort_values(['uuid','NOANNEE']).groupby('uuid')['Psychological disorders_binary'].diff(1)

In [ ]:
df_sample[['PRESTATIONS_BRUTES_LCA','PRESTATIONS_BRUTES_AOS','CDS_diff','cds']].corr()

In [ ]:
df_sample.groupby('Psy_diff')['PRESTATIONS_BRUTES_LCA'].mean()

In [ ]:
def check_change(group, var):
    diff = group[var].diff()
    return diff
var = 'Pain_binary'
# Group by PatientID and apply the custom function
changes = df_sample.groupby('uuid').apply(lambda x: check_change(x, var)).reset_index()

# Patients who changed from 0 to 1
increased_status = changes.loc[changes[var] == 1, 'uuid'].unique()

# Patients who changed from 1 to 0
decreased_status = changes.loc[changes[var] == -1, 'uuid'].unique()

# Filter the DataFrame for each case
df_increased = df_sample[df_sample['uuid'].isin(increased_status)]
df_decreased = df_sample[df_sample['uuid'].isin(decreased_status)]

In [ ]:
df_final['mammo_last_invite'] = df_final[['numerodossier','mammo']].groupby('numerodossier').diff()['mammo']

In [ ]:
df_sample.groupby('')

### 3. Using XGBoost

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
%%time
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval

# Choose hyperparameter domain to search over
space = {
        'max_depth':hp.choice('max_depth', np.arange(2, 30, 2, dtype=int)),
        'colsample_bytree':hp.quniform('colsample_bytree', 0.3, 1.01, 0.1),
        'min_child_weight':hp.choice('min_child_weight', np.arange(1, 30, 1, dtype=int)),
        'subsample':        hp.quniform('subsample', 0.3, 1.01, 0.1),
        'learning_rate':    hp.choice('learning_rate',    np.arange(0.1, 1.01, 0.1)),
        'gamma': hp.quniform('gamma', 0.1, 5, 0.2),
    
        'objective':'reg:squarederror',
        'eval_metric': 'rmse',
    }

def score(params, n_folds=5):
    _X_coords_sample = X_coords.sample(frac = 0.05, random_state=333).sort_index()
    _y_sample = y[y.index.isin(_X_coords_sample.index)]
    #Cross-validation
    d_train = xgboost.DMatrix(_X_coords_sample,_y_sample)
    
    cv_results = xgboost.cv(params, d_train, nfold = n_folds, num_boost_round=500,
                        early_stopping_rounds = 10, metrics = 'rmse', seed = 0)
    
    loss = min(cv_results['test-rmse-mean'])
    
    return loss


def optimize(trials, space, n_evals):
    
    best = fmin(score, space, algo=tpe.suggest, max_evals=n_evals,trials=trials,
                rstate=np.random.default_rng(333))#Add seed to fmin function
    return best

In [ ]:
def interpretable_ml_shap_viz_gpt(df, model, X, model_folder, include_spatial):
    model_directory = res_folder / model_folder
    
    explainer_shap = shap.TreeExplainer(model)
    shap_values = explainer_shap(X)
    shap_interaction_values = explainer_shap.shap_interaction_values(X)
    
    # Shapley summary
    shap.summary_plot(shap_values, feature_names=X.columns, show=False)
    plt.savefig(model_directory / 'shap_summary.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # Shapley interaction values
    shap.summary_plot(shap_interaction_values, X, max_display=16,
                      feature_names=X.columns, show=False, plot_type="compact_dot")
    plt.savefig(model_directory / 'shap_summary_interaction_values.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # Shapley dependence plots
    dependence_dir = model_directory / 'Dependence plots'
    if not os.path.exists(dependence_dir):
        os.makedirs(dependence_dir)
    
    for name in X.columns:
        shap.dependence_plot(name, shap_values.values, X, display_features=X, show=False)
        plt.savefig(dependence_dir / f'shap_dependence_{name}.png', dpi=300, bbox_inches='tight')
        plt.close()

    # Calculate the number of rows and columns for the subplots
    n_columns = len(X.columns)
    n_rows = int(np.ceil(n_columns / 3))

    # Maps of SHAP variables
    if include_spatial:
        fig, ax = plt.subplots(n_rows, 3, figsize=(15, 15 * n_rows / 3))
        ax = ax.ravel()

        for j in range(n_columns):
            df.plot(ax=ax[j], column=shap_values.values[:, j], legend=True, markersize = 1,
                    vmin=-shap_values.values[:, j].max(), vmax=shap_values.values[:, j].max(), cmap=shap.plots.colors.red_white_blue, legend_kwds={'shrink':0.5})
            ax[j].set_title("SHAP for\n" + str(X.columns[j]), fontsize=10)
            ax[j].set_axis_off()

        # Hide remaining unused subplots
        for j in range(n_columns, n_rows * 3):
            ax[j].set_axis_off()

        plt.tight_layout()
        plt.savefig(model_directory / 'maps_shap_variables.png', dpi=300, bbox_inches='tight')
        plt.close()

        # Map of location effects
        fig, ax = plt.subplots(dpi=300)

        df.plot(ax=ax, column=shap_values.values[:, -1] + shap_values.values[:, -2],
                legend=True, vmin=-(shap_values.values[:, -1] + shap_values.values[:, -2]).max(), vmax=(shap_values.values[:, -1] + shap_values.values[:, -2]).max(), figsize=(15, 8), markersize = 1,
                cmap=shap.plots.colors.red_white_blue)
        plt.title("Location Effect on Cluster persistence\n(SHAP values of geographic coordinates)\n", fontsize=8)
        plt.axis('off')
        plt.savefig(model_directory/ 'maps_shap_location.png', dpi=300, bbox_inches='tight')
        plt.close()

    # Variable importance
    xgboost.plot_importance(model)
    plt.title("xgboost.plot_importance(model)")
    plt.savefig(model_directory / 'importance_plot_default.png', dpi=300, bbox_inches='tight')
    plt.close()

    xgboost.plot_importance(model, importance_type="cover")
    plt.title('xgboost.plot_importance(model, importance_type="cover")')
    plt.savefig(model_directory / 'importance_plot_cover.png', dpi=300, bbox_inches='tight')
    plt.close()

    # Force plots
    shap.force_plot(explainer_shap.expected_value, shap_values.values[1, :], X.iloc[1, :], show=False, matplotlib=True)
    plt.savefig(model_directory / 'force_plot.png', dpi=300, bbox_inches='tight')
    plt.close()


def train_xgboost(gwr_df, X_eq, y_col, optimize, space, n_evals, file_prefix, include_spatial=True):
    model_directory = res_folder / file_prefix
    if not os.path.exists(model_directory):
        os.makedirs(model_directory)
    if include_spatial:
        X_coords = gwr_df[X_eq + ['E', 'N']]
    else:
        X_coords = gwr_df[X_eq]
    
    y = gwr_df[y_col]

    X_train, X_test, y_train, y_test = train_test_split(X_coords, y, 
                                                        test_size=0.2, 
                                                        random_state=0)
     ### Hyperparameter tuning
    if not os.path.exists(model_directory/'best_params.pkl'):

        trials = Trials()
        best_params = optimize(trials, space, n_evals=n_evals)

        # Return the best parameters
        best_params = space_eval(space, best_params)

        with open(model_directory / 'trials.pkl', 'wb') as f:
            pickle.dump(trials, f)

        with open(model_directory / 'best_params.pkl', 'wb') as f:
            pickle.dump(best_params, f)
    else:
        best_params = np.load(model_directory / 'best_params.pkl', allow_pickle=True)

        with open(model_directory/ 'trials.pkl', 'rb') as f:
            trials = pickle.load(f)


    ### Train model
    d_train = xgboost.DMatrix(X_train, label=y_train)
    d_test = xgboost.DMatrix(X_test, label=y_test)
    d_all = xgboost.DMatrix(X_coords, label=y)

    final_model = xgboost.train(best_params,  d_train, num_boost_round=500, evals=[(d_train, "train"), (d_test, "test")], 
                                verbose_eval=False, early_stopping_rounds=10)

    y_pred_t = final_model.predict(d_test)

    print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_t)))
    print("R2:", r2_score(y_test, y_pred_t))

    data_test = xgboost.DMatrix(X_test, y_test)
    data = xgboost.DMatrix(X_train, y_train)
    data_all = xgboost.DMatrix(X_coords)
    
    ### Fit model or retrieve it
    if not os.path.exists(model_directory/'model.pkl'):
        final_model = xgboost.train(best_params, data, num_boost_round=5000, verbose_eval=False,
                                evals=[(data_test, "Test")], early_stopping_rounds=10)
        with open(model_directory/'model.pkl', 'wb') as f:
            pickle.dump(final_model, f)
    else:
        final_model = np.load(model_directory / 'model.pkl', allow_pickle=True)

    y_pred = final_model.predict(data_all)
    print('RMSE:', np.sqrt(mean_squared_error(y, y_pred)))
    print('R2: ', r2_score(y, y_pred))
    
    interpretable_ml_shap_viz_gpt(gwr_df, final_model, X_coords, file_prefix, include_spatial)

    return final_model


In [ ]:
y = data_2017['PRESTATIONS_TOTAL']
X_eq0 = ['NBAGE','DEDUCTIBLE_above_500','SEX_F','CANTON_NAME_Genève','CANTON_NAME_Vaud','CANTON_NAME_Valais','ssep2']
X_coords = data_2017[X_eq0 + ['E','N']]

In [ ]:
import pickle

In [ ]:
# data_2017 = gpd.GeoDataFrame(data_2017, crs = 2056, geometry = data_2017['geometry'] )
# data_2018 = gpd.GeoDataFrame(data_2018, crs = 2056, geometry = data_2018['geometry'] )
# data_2019 = gpd.GeoDataFrame(data_2019, crs = 2056, geometry = data_2019['geometry'] )
# data_2020 = gpd.GeoDataFrame(data_2020, crs = 2056, geometry = data_2020['geometry'] )
# data_2021 = gpd.GeoDataFrame(data_2021, crs = 2056, geometry = data_2021['geometry'] )

In [ ]:
train_xgboost(data_2017, X_eq0, 'PRESTATIONS_TOTAL', optimize, space, n_evals=3000, file_prefix='Model 1')

In [ ]:
explainer_shap = shap.TreeExplainer(final_model)
shap_values = explainer_shap(X_coords)
shap_interaction_values = explainer_shap.shap_interaction_values(X_coords)

# Calculate the number of rows and columns for the subplots
n_columns = len(X_coords.columns)
n_rows = int(np.ceil(n_columns / 3))

fig, ax = plt.subplots(n_rows, 3, figsize=(15, 15 * n_rows / 3))
ax = ax.ravel()

for j in range(n_columns):
    data_2017.plot(ax=ax[j], column=shap_values.values[:, j], legend=True, markersize = 1,
            vmin=-shap_values.values[:, j].max(), vmax=shap_values.values[:, j].max(), cmap=shap.plots.colors.red_white_blue, legend_kwds={'shrink':0.5})
    ax[j].set_title("SHAP for\n" + str(X_coords.columns[j]), fontsize=10)
    ax[j].set_axis_off()

# Hide remaining unused subplots
for j in range(n_columns, n_rows * 3):
    ax[j].set_axis_off()

plt.tight_layout()
plt.show()
plt.close()

In [ ]:
with open(model_directory/'trials.pkl', 'rb') as f:
    trials = pickle.load(f)
trials = pd.DataFrame.from_dict(trials.results)
trials = trials.reset_index()

In [ ]:
trials['rolling_loss'] = trials['loss'].rolling(100).mean()
trials.plot(x = 'index', y = 'rolling_loss')

In [ ]:
X_coords = data_2017[X_eq0 + ['E', 'N']]
y = data_2017['treatment']

# X_train, X_test, y_train, y_test = train_test_split(X_coords, y, 
#                                                     test_size=0.2, 
#                                                     random_state=0)
# ### Hyperparameter tuning
# trials = Trials()
# best_params = optimize(trials, space, n_evals=500)

# # Return the best parameters
# best_params = space_eval(space, best_params)

# with open(file_prefix + '_trials.pkl', 'wb') as f:
#     pickle.dump(trials, f)

# with open(file_prefix + '_best_params.pkl', 'wb') as f:
#     pickle.dump(best_params, f)

# best_params = np.load(file_prefix + '_best_params.pkl', allow_pickle=True)

# with open(file_prefix + '_trials.pkl', 'rb') as f:
#     trials = pickle.load(f)